# Twitter Airline Sentiment Analysis

The data is taken from [kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data?select=Tweets.csv)

![NumPy](https://img.shields.io/badge/numpy-%23013243.svg?style=for-the-badge&logo=numpy&logoColor=white)